In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\Admin\\MLProjects\\Text-Summarizer-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [4]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [5]:
# import os
# import requests
# import zipfile
# from textSummarizer.logging import logger
# from textSummarizer.utils.common import get_size


# class DataIngestion():
#     def __init__(self, config: DataIngestionConfig):
#         self.config = config

#     def download_file(self,):
#         if not os.path.exists(self.config.local_data_file):
#             response = requests.get(self.config.source_URL)

#             if response.status_code == 200:
#                 with open(self.config.local_data_file, "wb") as file:
#                     file.write(response.content)

#                 logger.info(
#                     f"Download successful with the following headers: \n{response.headers}")

#         else:
#             logger.info(
#                 f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

#     def extract_zip_file(self):
#         """
#         zip_file_path: str
#         Extracts the zip file into the data directory
#         Function returns None
#         """
#         unzip_path = self.config.unzip_dir
#         os.makedirs(unzip_path, exist_ok=True)
#         with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
#             zip_ref.extractall(unzip_path)

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-06 15:22:00,576: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-06 15:22:00,588: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-06 15:22:00,590: INFO: common: created directory at : artifacts]
[2023-09-06 15:22:00,593: INFO: common: created directory at : artifacts/data_ingestion]
[2023-09-06 15:22:04,307: INFO: 2114368287: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2312:17955E:8E26C:C77DE:64F84B2E
Accept-Ranges: bytes
Date: Wed, 06 Sep 2023 09:52:03 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4744-BOM
X-Cache: H